# Get lugar de ocurrencia

#### **Objectivo**
* **Objectivo General**: Predecir la frecuencia de los accidentes de vehículo a lo largo del tiempo. Y responder la pregunta: : ¿Qué factores externos a las características del cliente afectan la frecuencia de los siniestros en auto?
* **Objectivo Especifico del Notebook**: Realizar Notebook para identicar la longitud y la latitud. 

In [3]:
pip install unidecode

  Using cached Unidecode-1.3.6-py3-none-any.whl (235 kB)
Note: you may need to restart the kernel to use updated packages.


In [38]:
# Importamos las librerias a utilizar
import pandas as pd
import numpy as np
import re
import unidecode
import time
from geopy.geocoders import GoogleV3

Clean the location dataset

In [17]:
# Cargamos
df = pd.read_excel("data/dataset_cleaned.xlsx")
df.columns

Index(['Codigo_Siniestro', 'Codigo_Cliente', 'Fe_Ocurrencia', 'Codigo_Poliza',
       'Cd_Provincia', 'Cd_Ciudad', 'De_Lugar_Ocurrencia',
       'Nu_Siniestro_Relacionado', 'In_Fraude', 'De_Causa_Siniestro',
       'De_Causa_Detalle', 'De_Provincia', 'De_Ciudad', 'Hora_Ocurrencia',
       'Relacion_Reclamante', 'Evento_Catastrofico', 'Tp_Documento', 'Cd_Sexo',
       'Cd_Estado_Civil', 'Rango_Edad', 'Meses_Desde_Registro',
       'In_Lista_Negra', 'Cd_Pais_Nacimiento', 'Cd_Nacionalidad',
       'In_Inactivo', 'Fe_Inactivo', 'In_Vip', 'Cd_Provincia_Nacimiento',
       'De_Act_Economica_Adicional', 'Cd_Sucursal_Envio', 'De_Sector',
       'De_Ocupacion', 'De_Actividad', 'De_Ingreso_Anual', 'De_Profesion',
       'De_Tipo_Persona', 'Tipo_Vehiculo', 'Color', 'Marca', 'Modelo', 'Año',
       'Tipo_Transmision', 'ano', 'Clase', 'Cantidad_Parque_Vehicular',
       'Tipo_Dia', 'dia_feriado'],
      dtype='object')

In [18]:
# Convertimos a minuscula nuestra base columna objectrivo
df["MOD_Lugar_Ocurrencia"] = df["De_Lugar_Ocurrencia"].str.lower()

In [19]:
def remover_acentos(x):
    """Creamos la funcion para remover los acentos"""
    if isinstance(x, str):
        return unidecode.unidecode(x)
    else:
        return x

# Aplicar la función a la columna "DE_LUGAR_OCURRENCIA"
df["MOD_Lugar_Ocurrencia"] = df["MOD_Lugar_Ocurrencia"].apply(remover_acentos)
df

,Codigo_Siniestro,Codigo_Cliente,Fe_Ocurrencia,Codigo_Poliza,Cd_Provincia,Cd_Ciudad,De_Lugar_Ocurrencia,Nu_Siniestro_Relacionado,In_Fraude,De_Causa_Siniestro,...,Marca,Modelo,Año,Tipo_Transmision,ano,Clase,Cantidad_Parque_Vehicular,Tipo_Dia,dia_feriado,MOD_Lugar_Ocurrencia
0,1,7668,2015-05-29,1,29.0,1.0,Siniestro reportado vía cover.,,0,Accidente/Choque,...,TOYOTA,CAMRY,2001.0,CE - Automatico,2015,Automovil,773019,Weekday,0,siniestro reportado via cover.
1,2,23643,2016-02-12,2,29.0,1.0,C/ MAYAGUEZ URB. VILLAS DUMIT,,0,Accidente/Choque,...,KIA,SPORTAGE,2015.0,4x4 - Automatico,2016,Jeep,386706,Weekday,0,c/ mayaguez urb. villas dumit
2,3,29730,2016-02-09,3,1.0,1.0,Barahora,,0,Responsabilidad Civil Personas o Cosas,...,DAIHATSU,DELTA,2007.0,Sincronico,2016,Carga,405292,Weekday,0,barahora
3,4,14301,2016-02-15,4,1.0,1.0,Arroyo Hondo II,,0,Robo o Hurto Accesorios/ Partes o Piezas,...,NaN,NaN,NaN,NaN,2016,Jeep,386706,Weekday,0,arroyo hondo ii
4,5,24144,2016-02-03,5,1.0,1.0,Avenida Gustavo Mejia Ricart,,0,Accidente/Choque,...,TOYOTA,PRADO,2013.0,%Sincronico,2016,Jeep,386706,Weekday,0,avenida gustavo mejia ricart
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18738,18702,192804,2023-04-04,11379,1.0,1.0,AUTOPISTA DUARTE,,0,Rotura de Vidrios y/o Parabrisas,...,JEEP,WRANGLER,2023.0,Renegade 4x4 - Automatico,2023,Jeep,653866,Weekday,0,autopista duarte
18739,18703,181905,2023-06-03,11470,1.0,1.0,LOS MULTIS LOS ALCARRIZOS,,0,Rotura de Vidrios y/o Parabrisas,...,ISUZU,D-MAX,2015.0,Automatico,2023,Jeep,653866,Weekend,0,los multis los alcarrizos
18740,18704,182109,2023-05-22,11471,1.0,1.0,AUTOPISTA DUARTE,,0,Rotura de Vidrios y/o Parabrisas,...,HYUNDAI,TUCSON,2014.0,GL - Automatico,2023,Jeep,653866,Weekday,0,autopista duarte
18741,18705,302838,2023-06-01,11472,1.0,1.0,CALLE CAMINO CHIQUITO,,0,Accidente/Choque,...,TOYOTA,HILUX,2023.0,Cabina Sencilla Pick-Up 4x4 - Sincronico,2023,Jeep,653866,Weekday,0,calle camino chiquito


Select only relevant features

In [24]:
# Seleccionamos las variables importantes
df2 = df[['Codigo_Siniestro','Codigo_Cliente','Fe_Ocurrencia','Codigo_Poliza',
         'Cd_Provincia','Cd_Ciudad','De_Lugar_Ocurrencia','MOD_Lugar_Ocurrencia',
         'De_Provincia','De_Ciudad']]
df2

,Codigo_Siniestro,Codigo_Cliente,Fe_Ocurrencia,Codigo_Poliza,Cd_Provincia,Cd_Ciudad,De_Lugar_Ocurrencia,MOD_Lugar_Ocurrencia,De_Provincia,De_Ciudad
0,1,7668,2015-05-29,1,29.0,1.0,Siniestro reportado vía cover.,siniestro reportado via cover.,Distrito Nacional,Santo Domingo
1,2,23643,2016-02-12,2,29.0,1.0,C/ MAYAGUEZ URB. VILLAS DUMIT,c/ mayaguez urb. villas dumit,,
2,3,29730,2016-02-09,3,1.0,1.0,Barahora,barahora,,
3,4,14301,2016-02-15,4,1.0,1.0,Arroyo Hondo II,arroyo hondo ii,Distrito Nacional,Santo Domingo
4,5,24144,2016-02-03,5,1.0,1.0,Avenida Gustavo Mejia Ricart,avenida gustavo mejia ricart,,
...,...,...,...,...,...,...,...,...,...,...
18738,18702,192804,2023-04-04,11379,1.0,1.0,AUTOPISTA DUARTE,autopista duarte,,
18739,18703,181905,2023-06-03,11470,1.0,1.0,LOS MULTIS LOS ALCARRIZOS,los multis los alcarrizos,,
18740,18704,182109,2023-05-22,11471,1.0,1.0,AUTOPISTA DUARTE,autopista duarte,,
18741,18705,302838,2023-06-01,11472,1.0,1.0,CALLE CAMINO CHIQUITO,calle camino chiquito,,


In [ ]:
def remove_special_characters(text, remove_digits=False):
    """Funcion para remover los caracteres especiales"""
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', text)
    return text

# Asegurarse de que todos los valores en la columna sean cadenas
df2["MOD_Lugar_Ocurrencia"] = df2["MOD_Lugar_Ocurrencia"].astype(str)

# Aplicar la función a la columna "DE_LUGAR_OCURRENCIA"
df2["MOD_Lugar_Ocurrencia"] = df2["MOD_Lugar_Ocurrencia"].apply(remove_special_characters)

In [ ]:
def categorizar_lugar(lugar):
    """Creamos una columna, para identificar donde fue el siniestro"""
    if "avenida" in lugar.lower() or "av" in lugar.lower():
        return "Avenida"
    elif "calle" in lugar.lower():
        return "Calle"
    elif "autopista" in lugar.lower() or "aut" in lugar.lower():
        return "Autopista"
    else:
        return "Otro"

df2["Categoria_Lugar"] = df2["MOD_Lugar_Ocurrencia"].apply(categorizar_lugar)
df2["Categoria_Lugar"].value_counts()

In [28]:
df2

,Codigo_Siniestro,Codigo_Cliente,Fe_Ocurrencia,Codigo_Poliza,Cd_Provincia,Cd_Ciudad,De_Lugar_Ocurrencia,MOD_Lugar_Ocurrencia,De_Provincia,De_Ciudad,Categoria_Lugar
0,1,7668,2015-05-29,1,29.0,1.0,Siniestro reportado vía cover.,siniestro reportado via cover,Distrito Nacional,Santo Domingo,Otro
1,2,23643,2016-02-12,2,29.0,1.0,C/ MAYAGUEZ URB. VILLAS DUMIT,c mayaguez urb villas dumit,,,Otro
2,3,29730,2016-02-09,3,1.0,1.0,Barahora,barahora,,,Otro
3,4,14301,2016-02-15,4,1.0,1.0,Arroyo Hondo II,arroyo hondo ii,Distrito Nacional,Santo Domingo,Otro
4,5,24144,2016-02-03,5,1.0,1.0,Avenida Gustavo Mejia Ricart,avenida gustavo mejia ricart,,,Avenida
...,...,...,...,...,...,...,...,...,...,...,...
18738,18702,192804,2023-04-04,11379,1.0,1.0,AUTOPISTA DUARTE,autopista duarte,,,Autopista
18739,18703,181905,2023-06-03,11470,1.0,1.0,LOS MULTIS LOS ALCARRIZOS,los multis los alcarrizos,,,Otro
18740,18704,182109,2023-05-22,11471,1.0,1.0,AUTOPISTA DUARTE,autopista duarte,,,Autopista
18741,18705,302838,2023-06-01,11472,1.0,1.0,CALLE CAMINO CHIQUITO,calle camino chiquito,,,Calle


In [41]:
# Definimos la API key (clave) oculta. Deberías tener una API key válida para utilizar el servicio de geolocalización.
api_key = 'hidden_key'

# Creamos una instancia del objeto geolocalizador de la API de Google Maps
geolocator = GoogleV3(api_key=api_key)

# Creamos listas vacías para almacenar las coordenadas de latitud y longitud
latitude_list = []
longitude_list = []

# Realizamos la geocodificación para cada dirección en la columna 'MOD_Lugar_Ocurrencia'
for address in df2['MOD_Lugar_Ocurrencia']:
    location = geolocator.geocode(address, components={"country":"DO"}, region="DO")
    if location:
        latitude_list.append(location.latitude)
        longitude_list.append(location.longitude)
    else:
        latitude_list.append(None)
        longitude_list.append(None)

df2['latitude'] = latitude_list
df2['longitude'] = longitude_list

print(df2)

       Codigo_Siniestro  Codigo_Cliente Fe_Ocurrencia  Codigo_Poliza  \
0                     1            7668    2015-05-29              1   
1                     2           23643    2016-02-12              2   
2                     3           29730    2016-02-09              3   
3                     4           14301    2016-02-15              4   
4                     5           24144    2016-02-03              5   
...                 ...             ...           ...            ...   
18738             18702          192804    2023-04-04          11379   
18739             18703          181905    2023-06-03          11470   
18740             18704          182109    2023-05-22          11471   
18741             18705          302838    2023-06-01          11472   
18742             18706          114156    2023-05-06          11311   

       Cd_Provincia  Cd_Ciudad  \
0              29.0        1.0   
1              29.0        1.0   
2               1.0        1.0   

C:\Users\Aneur\AppData\Local\Temp\ipykernel_7412\835576196.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['latitude'] = latitude_list
C:\Users\Aneur\AppData\Local\Temp\ipykernel_7412\835576196.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['longitude'] = longitude_list


In [92]:
df2_copy=df2.copy()

In [93]:
df2_copy

,Codigo_Siniestro,Codigo_Cliente,Fe_Ocurrencia,Codigo_Poliza,Cd_Provincia,Cd_Ciudad,De_Lugar_Ocurrencia,MOD_Lugar_Ocurrencia,De_Provincia,De_Ciudad,Categoria_Lugar,latitude,longitude
0,1,7668,2015-05-29,1,29.0,1.0,Siniestro reportado vía cover.,siniestro reportado via cover,Distrito Nacional,Santo Domingo,Otro,18.735693,-70.162651
1,2,23643,2016-02-12,2,29.0,1.0,C/ MAYAGUEZ URB. VILLAS DUMIT,c mayaguez urb villas dumit,,,Otro,18.735693,-70.162651
2,3,29730,2016-02-09,3,1.0,1.0,Barahora,barahora,,,Otro,18.212081,-71.102408
3,4,14301,2016-02-15,4,1.0,1.0,Arroyo Hondo II,arroyo hondo ii,Distrito Nacional,Santo Domingo,Otro,18.486257,-69.928257
4,5,24144,2016-02-03,5,1.0,1.0,Avenida Gustavo Mejia Ricart,avenida gustavo mejia ricart,,,Avenida,18.470057,-69.945025
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18738,18702,192804,2023-04-04,11379,1.0,1.0,AUTOPISTA DUARTE,autopista duarte,,,Autopista,18.490774,-69.982795
18739,18703,181905,2023-06-03,11470,1.0,1.0,LOS MULTIS LOS ALCARRIZOS,los multis los alcarrizos,,,Otro,18.524213,-70.020341
18740,18704,182109,2023-05-22,11471,1.0,1.0,AUTOPISTA DUARTE,autopista duarte,,,Autopista,18.490774,-69.982795
18741,18705,302838,2023-06-01,11472,1.0,1.0,CALLE CAMINO CHIQUITO,calle camino chiquito,,,Calle,18.493735,-69.937733


In [110]:
# Vemos los valores null
df2_copy[df2_copy.longitude.isnull()]

,Codigo_Siniestro,Codigo_Cliente,Fe_Ocurrencia,Codigo_Poliza,Cd_Provincia,Cd_Ciudad,De_Lugar_Ocurrencia,MOD_Lugar_Ocurrencia,De_Provincia,De_Ciudad,Categoria_Lugar,latitude,longitude
5711,5698,113322,2017-12-25,3905,1.0,1.0,Calle Francisco Villa Espesa,calle francisco villa espesa,,,Calle,NaN,NaN


Vemos que hay varios valores nulos, por lo que reemplazaremos estos casos de la siguiente forma:

In [124]:
# Diccionario que contiene coordenadas de latitud para ciertas direcciones con valores nulos en el DataFrame
latitude_null = {"casa firo": 19.37670771764009, 
"calle duarte frente a boca marina": 18.4485165127879, 
"coun bar": 18.487657428246784, 
"calle francisco villa espesa ": 18.487237023619702,
"calle francisco  villa espesa": 18.487237023619702,
"hotel majestic ": 19.794008615020143, 
"6 de noviembre": 18.430719619748526, 
"calle duarte proximo a boca marina": 18.4485165127879, 
"palacio de justicia": 18.467645167631986,
"siniestro reportado via cover": 18.489610429487747  # Esta direccion salia en Yamasa, colocamos Santo Domingo
                }

longitude_null = {"casa firo": -69.84757036314564,
"calle duarte frente a boca marina": -69.60169274661185,
"coun bar": -69.93175667080567,
"calle francisco villa espesa ": -69.91355485662457,
"calle francisco  villa espesa": -69.91355485662457,
"hotel majestic ": -70.70329012883053,
"6 de noviembre": -70.05841437694016,
"calle duarte proximo a boca marina": -69.60169274661185,
"palacio de justicia": -69.89393412565127,
"siniestro reportado via cover": -69.93267190991932  # Esta direccion salia en Yamasa, colocamos Santo Domingo
                }

In [125]:
# Agregamos nuevas columnas 'latitude2' y 'longitude2' al DataFrame df2_copy
# Mapeamos las direcciones de 'MOD_Lugar_Ocurrencia' con las coordenadas de latitud y longitud de los diccionarios
df2_copy['latitude2'] = df2_copy['MOD_Lugar_Ocurrencia'].map(latitude_null)
df2_copy['longitude2'] = df2_copy['MOD_Lugar_Ocurrencia'].map(longitude_null)

In [126]:
# Usamos np.where para llenar los valores nulos en 'latitude2' con valores de 'latitude' si están nulos
df2_copy['latitude2'] = np.where(df2_copy['latitude2'].isnull(), df2_copy['latitude'], df2_copy['latitude2'])

# Usamos np.where para llenar los valores nulos en 'longitude2' con valores de 'longitude' si están nulos
df2_copy['longitude2'] = np.where(df2_copy['longitude2'].isnull(), df2_copy['longitude'], df2_copy['longitude2'])

In [127]:
# Eliminamos las columnas 'latitude' y 'longitude' del DataFrame df2_copy
df2_copy.drop(columns=['latitude', 'longitude'], inplace=True)

# Renombramos las columnas 'latitude2' y 'longitude2' a 'latitude' y 'longitude'
df2_copy.rename(columns={'latitude2':'latitude', 'longitude2':'longitude'}, inplace=True)

In [128]:
df2_copy.isnull().sum()

Codigo_Siniestro          0
Codigo_Cliente            0
Fe_Ocurrencia             0
Codigo_Poliza             0
Cd_Provincia              1
Cd_Ciudad                 1
De_Lugar_Ocurrencia     212
MOD_Lugar_Ocurrencia      0
De_Provincia              0
De_Ciudad                 0
Categoria_Lugar           0
latitude                  0
longitude                 0
dtype: int64

Para poder unificar esta data con nuestra data del clima y la temperatura promedio por dia, necesitamos una columna de `PROVINCIA` que sea fiable. Por lo cual, con las columnas obtenidas de `latitude` y `longitude`, planteamos la misma corrida con el API de Google Maps, esta vez para obtener la data geografica de la Provincia.

In [154]:
df3_copy = df2_copy.copy()

In [155]:
# Definimos una función llamada 'get_component' que obtiene un componente específico de una ubicación
def get_component(location, component_type):
    for component in location.raw['address_components']:
        if component_type in component['types']:
            return component['long_name']

# Creamos una lista vacía llamada 'state_list' para almacenar los nombres de las provincias
state_list = []

# Iteramos a través de las coordenadas de latitud y longitud en el DataFrame df3_copy
for i, j in zip(df3_copy["latitude"], df3_copy['longitude']):
    location = geolocator.reverse((i,j), timeout = 500,  exactly_one=True)
    state = get_component(location, 'administrative_area_level_1')
    state_list.append(state)

# Aplicamos funciones para remover acentos y convertir a mayúsculas en la columna 'PROVINCIA'
df3_copy['PROVINCIA'] = state_list
df3_copy['PROVINCIA'] = df3_copy['PROVINCIA'].apply(remover_acentos)
df3_copy['PROVINCIA'] = df3_copy['PROVINCIA'].str.upper()
df3_copy.head(5)

,Codigo_Siniestro,Codigo_Cliente,Fe_Ocurrencia,Codigo_Poliza,Cd_Provincia,Cd_Ciudad,De_Lugar_Ocurrencia,MOD_Lugar_Ocurrencia,De_Provincia,De_Ciudad,Categoria_Lugar,latitude,longitude,PROVINCIA
0,1,7668,2015-05-29,1,29.0,1.0,Siniestro reportado vía cover.,siniestro reportado via cover,Distrito Nacional,Santo Domingo,Otro,18.489610,-69.932672,DISTRITO NACIONAL
1,2,23643,2016-02-12,2,29.0,1.0,C/ MAYAGUEZ URB. VILLAS DUMIT,c mayaguez urb villas dumit,,,Otro,18.735693,-70.162651,SAN CRISTOBAL
2,3,29730,2016-02-09,3,1.0,1.0,Barahora,barahora,,,Otro,18.212081,-71.102408,BARAHONA
3,4,14301,2016-02-15,4,1.0,1.0,Arroyo Hondo II,arroyo hondo ii,Distrito Nacional,Santo Domingo,Otro,18.486257,-69.928257,DISTRITO NACIONAL
4,5,24144,2016-02-03,5,1.0,1.0,Avenida Gustavo Mejia Ricart,avenida gustavo mejia ricart,,,Avenida,18.470057,-69.945025,DISTRITO NACIONAL


In [157]:
df3_copy.isnull().sum()

Codigo_Siniestro          0
Codigo_Cliente            0
Fe_Ocurrencia             0
Codigo_Poliza             0
Cd_Provincia              1
Cd_Ciudad                 1
De_Lugar_Ocurrencia     212
MOD_Lugar_Ocurrencia      0
De_Provincia              0
De_Ciudad                 0
Categoria_Lugar           0
latitude                  0
longitude                 0
PROVINCIA                38
dtype: int64

In [194]:
df4_copy = df3_copy.copy()

In [195]:
# Reemplazamos la palabra " PROVINCE" por una cadena vacía en la columna 'PROVINCIA'
df4_copy['PROVINCIA'] = df4_copy['PROVINCIA'].str.replace(" PROVINCE", "")

In [196]:
# Filtramos el DataFrame df4 para mostrar las filas donde la columna 'PROVINCIA' tiene valores nulos
# Luego seleccionamos las columnas 'MOD_Lugar_Ocurrencia', 'PROVINCIA', 'longitude' y 'latitude' para estas filas
df4_copy[df4_copy.PROVINCIA.isnull()][['MOD_Lugar_Ocurrencia','PROVINCIA','longitude','latitude']]

,MOD_Lugar_Ocurrencia,PROVINCIA,longitude,latitude
1154,sierra del bahoruco,None,-71.416667,18.116667
1378,calle central,None,-70.482508,18.310618
1544,carretera casa de campo,None,-68.918855,18.420861
3635,calle 9 mirador isabela,None,-71.078747,19.889046
3801,la romana casa de campo,None,-68.918855,18.420861
4329,carretera camino a casa de campo,None,-68.918855,18.420861
4491,casa de campo romana,None,-68.918855,18.420861
6287,calle central,None,-70.482508,18.310618
6826,marbella juan dolio,None,-69.393723,18.427467
6900,camino hacia casa de campo,None,-68.918855,18.420861


In [197]:
# Diccionario que contiene valores para rellenar los valores nulos en la columna 'PROVINCIA'
fill_na_values = {"calle central": 'SANTO DOMINGO',
"calle central ": 'SANTO DOMINGO',
"casa de campo": 'LA ROMANA',
"calle freddy gaton arce": 'SANTO DOMINGO',
"boulevar de juan dolio sector playa juan dolio": 'SAN PEDRO DE MACORIS',
"avenida gaston fernanda deligne frente al malecon san pedro": 'SAN PEDRO DE MACORIS',
"casa campo": 'LA ROMANA',
"puente higuamo": 'SAN PEDRO DE MACORIS',
"proximo a la isabela aguiar ": 'SANTO DOMINGO',
"marbella juan dolio": 'SAN PEDRO DE MACORIS',
"marbella  juan dolio": 'SAN PEDRO DE MACORIS',
"la romana casa de campo": 'LA ROMANA',
"la isabela": 'PUERTO PLATA',
"casa de campo romana": 'LA ROMANA',
"carretera constanza  guayabal constanza": 'LA VEGA',
"carretera luperon sector la isabela puerto plata": 'PUERTO PLATA',
"carretera casa de campo": 'LA ROMANA',
"carretera camino a casa de campo": 'LA ROMANA',
"camino hacia casa de campo": 'LA ROMANA',
"calle jose roja": 'SAN PEDRO DE MACORIS',
"calle eugenio rodriguezavenida gsuon fernandez deligne": 'SAN PEDRO DE MACORIS',
"calle el valle": 'SANTO DOMINGO',
"calle 9 mirador isabela": 'PUERTO PLATA',
"sierra del bahoruco": 'PEDERNALES'}

In [198]:
# Mapeamos las direcciones en la columna 'MOD_Lugar_Ocurrencia' utilizando el diccionario 'fill_na_values'
# Los valores mapeados se agregan a una nueva columna 'PROVINCIA2'
df4_copy['PROVINCIA2'] = df4_copy['MOD_Lugar_Ocurrencia'].map(fill_na_values)

In [199]:
# Completamos los valores nulos.
df4_copy['PROVINCIA2'] = np.where(df4_copy['PROVINCIA2'].isnull(), df4_copy['PROVINCIA'], df4_copy['PROVINCIA2'])
df4_copy.drop(columns=['PROVINCIA'], inplace=True)
df4_copy.rename(columns={'PROVINCIA2':'PROVINCIA'}, inplace=True)

In [200]:
df4_copy.PROVINCIA.unique()

array(['DISTRITO NACIONAL', 'SAN CRISTOBAL', 'BARAHONA', 'SANTO DOMINGO',
       'LA ROMANA', 'SANTIAGO', 'MONSENOR NOUEL', 'LA ALTAGRACIA',
       'MONTE CRISTI', 'HERMANAS MIRABAL', 'PUERTO PLATA', 'EL SEIBO',
       'LA VEGA', 'PERAVIA', 'AZUA', 'PEDERNALES', 'SAN PEDRO DE MACORIS',
       'BAORUCO', 'SAMANA', 'MONTE PLATA', 'SANTIAGO RODRIGUEZ',
       'MARIA TRINIDAD SANCHEZ', 'SANCHEZ RAMIREZ', 'SAN JUAN',
       'VALVERDE', 'DUARTE', 'INDEPENDENCIA', 'HATO MAYOR', 'BAHORUCO',
       'ESPAILLAT', 'SAN JOSE DE OCOA', 'DAJABON', 'ELIAS PINA'],
      dtype=object)

In [201]:
df4_copy.isnull().sum()

Codigo_Siniestro          0
Codigo_Cliente            0
Fe_Ocurrencia             0
Codigo_Poliza             0
Cd_Provincia              1
Cd_Ciudad                 1
De_Lugar_Ocurrencia     212
MOD_Lugar_Ocurrencia      0
De_Provincia              0
De_Ciudad                 0
Categoria_Lugar           0
latitude                  0
longitude                 0
PROVINCIA                 0
dtype: int64

### Exportamos nuestra base de datos.

In [203]:
df4_copy.to_excel("data/lon_lat_data.xlsx", index=False)